Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
- взять в тренировочный набор турниры с dateStart из 2019 года;
- в тестовый — турниры с dateStart из 2020 года.

#### Читаем данные

In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

import scipy.sparse as sparse
import scipy.stats as stats
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder


In [2]:
players = pickle.load(open('chgk/players.pkl', 'rb'))
results = pickle.load(open('chgk/results.pkl', 'rb'))
tournaments = pickle.load(open('chgk/tournaments.pkl', 'rb'))

In [3]:
print('Общее число игроков:', len(players))
print('Общее число турниров:', len(tournaments))
print('Размер словаря результатов:', len(results))

Общее число игроков: 204063
Общее число турниров: 5528
Размер словаря результатов: 5528


#### Обработка данных

In [4]:
# Список id турниров для train и test
train_ids = []
test_ids = []

# 2019 в обучение, 2020 на тест
train_results = {}
test_results = {}

for tour_id, tour_data in tqdm(results.items()):
    if not tour_data:
        continue

    train_year_flag = tournaments[tour_id]['dateStart'].startswith('2019')
    test_year_flag = tournaments[tour_id]['dateStart'].startswith('2020')

    if not (test_year_flag or train_year_flag):
        continue

    # если в результатах турнира нет ни записей об ответах команд, ни о командах - то пропускаем такой турнир
    skip_flag = False
    for team_results in tour_data:
        if 'mask' in team_results and 'teamMembers' in team_results :
            if team_results['mask'] and team_results['teamMembers'] :
                continue
            else:
                skip_flag = True
                break
        else:
            skip_flag = True
            break

    if skip_flag:
        continue

    # Оставляем те команды, у которых число ответов равно числу вопросов турнира
    max_len_mask = max([len(team['mask']) for team in tour_data])
    data = [team for team in tour_data if len(team['mask']) == max_len_mask]

    if train_year_flag:
        train_ids.append(tour_id)
        train_results[tour_id] = data
    else:
        test_ids.append(tour_id)
        test_results[tour_id] = data

print('Количество турниров для обучающей выборки:', len(train_results))
print('Количество турниров для тестовой выборки:', len(test_results))

100%|██████████| 5528/5528 [00:01<00:00, 3811.61it/s] 

Количество турниров для обучающей выборки: 650
Количество турниров для тестовой выборки: 164


In [5]:
del results
del tournaments

In [6]:
# функция для замены символов в маске на 0
correct_mask = lambda x: x.replace('X', '0').replace('?', '0').replace('-', '0')

RESULT = []
for tour_id in tqdm(train_results.keys()):
    tour_results = train_results[tour_id]
    for team_tour_results in tour_results:
        mask = correct_mask(team_tour_results['mask'])
        team_id = team_tour_results['team']['id']
        players_list = [p['player']['id'] for p in team_tour_results['teamMembers']]
        position = team_tour_results['position']
        for question in range(len(mask)):
            question_id = str(tour_id) + f'_{question}'
            is_answered = int(mask[question])
            for player_id in players_list:
                RESULT.append([player_id, team_id, position, tour_id, question_id, is_answered])

# В результирующий датасет добавляем информацию по игрокам, командам,
# позициям команд, турнирам, вопросам в турнире, и был ли на них ответ
players_results_train = pd.DataFrame(RESULT,
                                     columns=['player_id', 'team_id', 'position', 'tournament_id', 'question_id',
                                              'is_answered'])
RESULT = []

for tour_id in tqdm(test_results.keys()):
    tour_results = test_results[tour_id]
    for team_tour_results in tour_results:
        mask = correct_mask(team_tour_results['mask'])
        team_id = team_tour_results['team']['id']
        players_list = [p['player']['id'] for p in team_tour_results['teamMembers']]
        position = team_tour_results['position']
        for question in range(len(mask)):
            question_id = str(tour_id) + f'_{question}'
            is_answered = int(mask[question])
            for player_id in players_list:
                RESULT.append([player_id, team_id, position, tour_id, question_id, is_answered])

players_results_test = pd.DataFrame(RESULT,
                                    columns=['player_id', 'team_id', 'position', 'tournament_id', 'question_id',
                                             'is_answered'])

100%|██████████| 164/164 [00:17<00:00,  9.45it/s]


In [7]:
players_results_test

,player_id,team_id,position,tournament_id,question_id,is_answered
0,30152,49804,1.0,4957,4957_0,1
1,30270,49804,1.0,4957,4957_0,1
2,27822,49804,1.0,4957,4957_0,1
3,28751,49804,1.0,4957,4957_0,1
4,27403,49804,1.0,4957,4957_0,1
...,...,...,...,...,...,...
3788399,154624,63129,7.0,6456,6456_37,0
3788400,224329,63129,7.0,6456,6456_37,0
3788401,165962,63129,7.0,6456,6456_38,1
3788402,154624,63129,7.0,6456,6456_38,1


In [8]:
# Подсчет уникальных игроков в обучающем датасете
player_ids_train = set()
for _, tour_data in train_results.items():
    # Цикл по результатам команд в турнире
    for result in tour_data:
        for player in result['teamMembers']:
            player_ids_train.add(player['player']['id'])

player_ids_train = list(player_ids_train)
num_of_players = len(player_ids_train)

print('Количество игроков в обучающем датасете:', num_of_players)

Количество игроков в обучающем датасете: 53775


Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
- повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
- в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
- для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.


Предположения:
- рейтинг игрока - это вероятность его ответить на случайный вопрос
- ответ, который дает команда равна, тому - что каждый из ее участников ответил на вопрос

Так как необходимо учесть силу каждого игрока, и сложность каждого из вопросов, логично - обучить логистическую регрессию, которая, учитывая факторы сложности вопроса и силу игрока, будет прогнозировать вероятность ответа на случайный вопрос. Таким образом - сила конкретного игрока и сложность конкретного вопроса, будут являтся весами логистической регрессии, полученных во время обучения.

Таким образом - в качестве преобразования данных для обучения можно взять OneHotEncoder, где каждый игрок и вопрос будут факторами для модели с своими весами.


$P(player_i) = \sigma(bias + W_{players}*X_{players} + W_{questions}*X_{questions})$ ,
где $player_i$ - i-ый игрок,
$W_{players}$ / $W_{questions}$ - сила игроков / сложность вопросов,
$X_{players}$ / $X_{questions}$ - игроки / вопросы.


In [9]:
# Трансформируем данные для обучения
players_encoder = OneHotEncoder()  # энкодер понадобится далее
questions_encoder = OneHotEncoder()  # энкодер понадобится далее\

player_dummies = players_encoder.fit_transform(players_results_train['player_id'].values.reshape(-1, 1))
question_dummies = questions_encoder.fit_transform(players_results_train['question_id'].values.reshape(-1, 1))

# Создаем датасет объединением one-hot представлений игроков и вопросов
X = sparse.hstack((player_dummies, question_dummies))
print(X.shape)

(15222736, 84603)


In [10]:
#Обучаем регрессию на получение вероятности ответа на вопрос.
clf = LogisticRegression(random_state=42)
clf.fit(X, players_results_train['is_answered'])

/Users/18843024/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [11]:
# Смещение
w0 = clf.intercept_
# Обученные веса игроков - силы игроков
player_weights = clf.coef_[0, :num_of_players]
# Обученные веса вопросов - сложности вопросов
question_weights = clf.coef_[0, num_of_players:]

Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
- предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
- в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров

Результаты турнира будут оцениваться по количеству отвеченных игроками вопросами.
Найдем вероятность ответа на случайный вопрос случайным игроком, без учета самого вопроса. Для этого можно взять сигмоиду и коэффиценты сил игроков (априорная вероятность ответа игрока на вопрос), и подставить значения.

$P(player_i) = \sigma(bias + [rating_i = W_{players}^TX_{players}])$ ,
где $player_i$ - i-ый игрок, $rating_i$ - рейтинг i-го игрока, $bias$ и $W$ - обучаемые регрессией параметры

Также, с учетом предположений стоит учесть, что вероятность ответить на вопрос - это вероятность того, что хотя бы один из игроков команды ответит на него:

$P(team)=1-\prod_{player_i \in team}(1 - P(player_i))$,

где $team$ - команда, где находится i-ый игрок

In [12]:
# Функция для подсчета вероятности ответа на вопрос, с учетом команды
def team_proba(team, player_ratings):
    result = 1
    for player in team:
        if player['player']['id'] in player_ratings.index:
            p = player_ratings[player['player']['id']]
        else:
            p = 0
        result *= (1 - p)
    return 1 - result

In [13]:
# Функция для подсчета корреляций

def correlation_count(test_results, p_rates):
    spearman = 0
    kendall = 0

    for tour_id in test_results.keys():
        # Пропускаем турнир если в результатах нет инфо о позиции команды
        if not 'position' in test_results[tour_id][0].keys():
            continue
        pred = np.zeros(len(test_results[tour_id]))
        true = np.zeros(len(test_results[tour_id]))

        for ind, result in enumerate(test_results[tour_id]):
            true[ind] = result['position']
            #Cчитаем по команде целиком
            pred[ind] = team_proba(result['teamMembers'], p_rates)

        spearman_current = stats.spearmanr(true, pred, nan_policy = 'omit')[0]
        kendall_current = stats.kendalltau(true, pred, nan_policy = 'omit')[0]

        if not (np.isnan(spearman_current) or np.isnan(kendall_current)):
            spearman += spearman_current
            kendall += kendall_current

    return spearman / len(test_results), kendall / len(test_results)

In [14]:
def sigmoid(bias, values):
    return 1 / (1 + np.exp(-(bias + values)))

players_ratings = pd.Series(np.zeros(num_of_players, dtype=float), index=player_ids_train)
# Индексы массива для выборки рейтингов игроков
players_ratings_idx = players_ratings.index.values.reshape(-1, 1)
idx = np.asarray(players_encoder.transform(players_ratings_idx).argmax(axis=1)).reshape(-1)
# Для каждого игрока рассчитаем вероятность без учета самого вопроса
players_ratings[players_ratings.index] = sigmoid(w0, player_weights[idx])

In [15]:
spearman_coef, kendall_coef = correlation_count(test_results, players_ratings)

print('Корреляция Спирмена test:', spearman_coef)
print('Корреляция Кендалла test:', kendall_coef)

Корреляция Спирмена test: -0.7744811972542103
Корреляция Кендалла test: -0.6184442499762459


In [16]:
#Выведем топ 10 игроков по версии модели.
def weights_idx_2_player_idx(ind):
    X = np.zeros((len(ind), num_of_players))
    for i in range(len(ind)):
        X[i, ind[i]] = 1
    return players_encoder.inverse_transform(X).reshape(-1)

top10_weight = (-player_weights).argsort()[:10]
top10_ids= weights_idx_2_player_idx(top10_weight)

print('Top 10 players:')
for i in range(len(top10_ids)):
    player = players[top10_ids[i]]
    weight_id = top10_weight[i]
    print(f"{i + 1}. {player['surname']} {player['name']} {player['patronymic']}: {player_weights[weight_id]}")

Top 10 players:
1. Мельникова Ольга Андреевна: 3.71346547663655
2. Семушин Иван Николаевич: 3.623982466445479
3. Брутер Александра Владимировна: 3.517658543418804
4. Савченков Михаил Владимирович: 3.514537931168285
5. Руссо Максим Михайлович: 3.4562933689108286
6. Спешков Сергей Леонидович: 3.3010689012035144
7. Салихов Максим Юрьевич: 3.299830024487427
8. Саксонов Антон Владимирович: 3.2561101863353707
9. Михайлов Кирилл Игоревич: 3.2398209305788033
10. Юнгер Мария Алексеевна: 3.2230727998523165


P.S. на EM не хватило ни ума, ни времени :(